In [ ]:
import cv2
import numpy as np
import torch
from detectron2.utils.visualizer import Visualizer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.structures import Boxes, BoxMode

from pathlib import Path

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None
        self.activations = None
        self.hook_handles = []

        # register hooks
        self.hook_handles.append(target_layer.register_forward_hook(self._save_activation))
        self.hook_handles.append(target_layer.register_backward_hook(self._save_gradient))

    def _save_activation(self, module, input, output):
        if output.ndim == 3:
            output = output.unsqueeze(0)
        self.activations = output.detach()

    def _save_gradient(self, module, grad_input, grad_output):
        grad = grad_output[0]
        if grad.ndim == 3:
            grad = grad.unsqueeze(0)
        self.gradients = grad.detach()

    def remove_hooks(self):
        for h in self.hook_handles:
            h.remove()

    def __call__(self, input_tensor, class_idx=None):
        # Forward
        output = self.model(input_tensor)
        
        # If it's a detection model, you'll need to pick a score or logit
        if class_idx is None:
            score = output[0]["instances"].scores[0]  # first detection
        else:
            # print("class_idx: ", class_idx)
            # print(output[0])
            score = output[0]["instances"].scores[int(class_idx)]

        # Backward
        self.model.zero_grad()
        score.backward(retain_graph=True)

        # Global-average-pool gradients
        weights = self.gradients.mean(dim=[2, 3], keepdim=True)  # [C,1,1]

        # Weighted sum of activations
        cam = (weights * self.activations).sum(dim=1, keepdim=True)
        cam = torch.relu(cam)
        
        # Normalize to [0,1]
        cam = cam.squeeze().cpu().numpy()
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
        return cam


# ---- Visualizer helper ----
def draw_predictions(img, outputs, metadata, gt_boxes, gt_classes):
    v = Visualizer(img[:, :, ::-1], metadata=metadata, scale=1.0)
    inst = outputs["instances"].to("cpu")
    # v = v.draw_instance_predictions(inst)

    # if gt_boxes is not None:
        # Draw GT boxes in red
    for box, cls in zip(gt_boxes, gt_classes):
        x1, y1, x2, y2 = map(int, box)
        cls_name = metadata.thing_classes[cls] if len(metadata.thing_classes) > 0 else str(cls)
        text = f"GT: {cls_name}"

        v.draw_box([x1, y1, x2, y2], edge_color=(0.0,1.0,0.0))
        v.draw_text(text, (x1, y1), color=(0.0,1.0,0.0))

    boxes = inst.pred_boxes.tensor.numpy()
    scores = inst.scores.tolist()
    classes = inst.pred_classes.tolist()

    for box, score, cls_id in zip(boxes, scores, classes):
        x0, y0, x1, y1 = box
        cls_name = metadata.thing_classes[cls_id] if len(metadata.thing_classes) > 0 else str(cls_id)
        text = f"Pred: {cls_name} {score:.2f}"

        v.draw_box([x0, y0, x1, y1], edge_color=(1.0, 0.0, 0.0))
        v.draw_text(text, (x0, y0), color=(1.0, 0.0, 0.0))

    vis_out = v.output
    drawn = vis_out.get_image()[:, :, ::-1]

    return drawn


def get_gt_from_dict(entry):
    gt_boxes, gt_classes = [], []
    for ann in entry["annotations"]:
        bbox = ann["bbox"]
        if ann["bbox_mode"] != BoxMode.XYXY_ABS:
            bbox = BoxMode.convert(bbox, ann["bbox_mode"], BoxMode.XYXY_ABS)
        gt_boxes.append(bbox)
        gt_classes.append(ann["category_id"])
    return gt_boxes, gt_classes  


# ---- Main routine for one image ----
def visualize_cam_and_bboxes(entry, model, gradcam, metadata, out_dir):
    img_path = entry["file_name"]
    h, w = entry["height"], entry["width"]
    img = cv2.imread(img_path)

    # Detectron2 input
    inputs = [{"image": torch.as_tensor(img.astype("float32").transpose(2, 0, 1)).cuda(),
               "height": h, "width": w}]
    with torch.no_grad():
        outputs = model(inputs)

    # Grad-CAM on top detection (if any)
    if len(outputs[0]["instances"]) > 0:
        score = outputs[0]["instances"].scores[0]
        cam_map = gradcam(inputs, score)
        cam_resized = cv2.resize(cam_map, (w, h))
        heatmap = cv2.applyColorMap(np.uint8(255 * cam_resized), cv2.COLORMAP_JET)
        overlay = (0.5 * heatmap + 0.5 * img).astype(np.uint8)
    else:
        overlay = img.copy()

    # GT boxes
    gt_boxes, gt_classes = get_gt_from_dict(entry)

    # print("GT boxes:", gt_boxes[:3])
    # print("GT classes:", gt_classes[:3])

    # Pred + GT
    bbox_vis = draw_predictions(img.copy(), outputs[0], metadata, gt_boxes, gt_classes)

    # Stack horizontally
    stacked = np.hstack([overlay, bbox_vis])

    # Save
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    out_path = Path(out_dir) / Path(img_path).name
    cv2.imwrite(str(out_path), stacked)
    print("Saved:", out_path)


In [44]:
import numpy as np
from sklearn import preprocessing

import _init_paths
from config import cfg, update_config

def update_cfg_with_args(cfg, arg_key, arg_value):
    cfg.defrost()

    arg_key = arg_key.upper()

    cfg.arg_key = arg_value

    cfg.freeze()


minmax_scaler = preprocessing.MinMaxScaler()

def minmax_norm(img):
    img = np.array([
        minmax_scaler.fit_transform(img[:, :, 0]),
        minmax_scaler.fit_transform(img[:, :, 1]),
        minmax_scaler.fit_transform(img[:, :, 2]),
    ])

    return np.transpose(img, (1, 2, 0)).astype(np.float32)


In [45]:
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog

from dataset.utils import register_patch_bin_dataset


cfg_path = "/workspace/project/configs/frcnn/frcnn.yaml"

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(
        "COCO-Detection/faster_rcnn_R_50_FPN_1x.yaml"
))

cfg.set_new_allowed(True)
cfg.defrost()
cfg.merge_from_file(cfg_path)

cfg.MODEL.WEIGHTS = "/workspace/project/record/debug2/result_single/frcnn_vis/model_final.pth"
cfg.MODEL.DEVICE = "cuda"

cfg.freeze()

predictor = DefaultPredictor(cfg)
model = predictor.model
model.eval()

target_layer = model.backbone.bottom_up.res5[-1]
gradcam = GradCAM(model, target_layer)

register_patch_bin_dataset(
        cfg.DATASETS.TEST[0],
        json_file=cfg.DATASETS.TEST_ANNO_DIR,
        img_root=cfg.DATASETS.IMG_DIR,
        extra_key=["patient_id"]
)

ds_dicts = DatasetCatalog.get(cfg.DATASETS.TEST[0])
metadata = MetadataCatalog.get(cfg.DATASETS.TEST[0])

out_dir = "./gradcam_results"

for i, entry in enumerate(ds_dicts):
        visualize_cam_and_bboxes(entry, model, gradcam, metadata, out_dir)

      

Config '/workspace/project/configs/frcnn/frcnn.yaml' has no VERSION. Assuming it to be compatible with latest v2.


/usr/local/lib/python3.12/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Instances(num_instances=6, image_height=1200, image_width=600, fields=[pred_boxes: Boxes(tensor([[ 253.8135,   87.3590,  362.0257,  197.7949],
        [ 235.2458,  296.0808,  351.4508,  405.0326],
        [ 233.0878,  978.2848,  347.1781, 1095.5038],
        [ 239.2220,  512.9576,  356.4888,  626.9459],
        [ 239.2888,  748.5310,  361.6597,  873.8732],
        [ 233.6026,  746.5156,  359.2216,  875.2394]])), scores: tensor([1.0000, 1.0000, 0.9999, 0.9993, 0.9765, 0.1430]), pred_classes: tensor([0, 0, 0, 0, 0, 1])])
Saved: gradcam_results/raw1_09-D2(30m)-4.JPG


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Instances(num_instances=16, image_height=1200, image_width=600, fields=[pred_boxes: Boxes(tensor([[  87.3732,  788.4876,  196.5059,  896.8377],
        [  94.6170,  307.0933,  205.9943,  419.1758],
        [  93.2583,   95.2293,  206.4622,  210.0879],
        [ 103.3276, 1019.2393,  206.1108, 1122.8838],
        [ 398.1685, 1017.9677,  499.3303, 1119.8344],
        [ 237.0203,  927.5140,  353.1707, 1040.3225],
        [ 259.8050,  689.9818,  362.6346,  791.3562],
        [ 400.4636,  801.8005,  500.0779,  903.9296],
        [ 266.6750,  248.3050,  362.9833,  344.5712],
        [ 252.0018,  477.6390,  362.0543,  594.1254],
        [ 404.1386,  351.4181,  510.1773,  453.1386],
        [ 389.8467,  561.6180,  501.6767,  673.8361],
        [  82.6073,  534.4041,  195.0396,  647.8557],
        [ 255.4333,   33.5069,  359.4947,  137.3125],
        [ 409.5235,  127.9151,  519.0797,  235.2624],
        [ 410.2410,  130.0424,  517.3726,  237.3010]])), scores: tensor([1.0000, 1.0000, 1.0000, 1.0

...

Instances(num_instances=13, image_height=1200, image_width=600, fields=[pred_boxes: Boxes(tensor([[ 245.9615,  707.2903,  367.3765,  832.0807],
        [ 236.4937,  999.9089,  357.8369, 1109.8191],
        [ 239.4290,  530.1555,  363.7381,  645.7405],
        [ 241.4951,  281.9655,  370.4108,  428.6078],
        [ 242.5217,  275.3396,  370.2727,  413.8956],
        [ 250.1685,  433.3153,  371.7965,  537.9316],
        [ 249.3507,    0.0000,  375.0056,  125.6609],
        [ 242.6943,  880.9062,  361.5560,  973.9417],
        [ 237.4040,  511.1281,  361.0320,  644.5662],
        [ 240.6844,  706.3695,  366.4760,  836.5470],
        [ 241.1780, 1002.4901,  357.3025, 1122.3986],
        [ 252.1496,  168.7305,  365.1082,  279.7702],
        [ 248.8627,  605.7621,  361.8766,  735.4530]])), scores: tensor([0.9443, 0.9371, 0.9321, 0.8431, 0.7506, 0.7067, 0.6327, 0.5428, 0.2924,
        0.1915, 0.1463, 0.1278, 0.0967]), pred_classes: tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])])
Saved: grad

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Saved: gradcam_results/raw2_33-D2(30m)-1.JPG
Instances(num_instances=17, image_height=1200, image_width=600, fields=[pred_boxes: Boxes(tensor([[ 101.8990, 1041.0977,  216.5434, 1153.6797],
        [ 269.6308,  949.0531,  387.7348, 1064.3497],
        [ 103.5645,   75.8946,  223.5147,  192.7368],
        [ 286.5324,  223.7887,  395.5362,  332.5936],
        [ 114.5212,  809.6195,  219.2520,  915.3647],
        [ 281.8674,  475.6057,  392.0549,  588.1415],
        [ 288.6986,   29.5602,  391.8347,  133.3664],
        [ 436.4666,  798.5922,  544.2048,  908.9470],
        [ 115.3022,  555.1224,  231.7346,  674.2962],
        [ 424.0859, 1038.2909,  532.4033, 1147.3600],
        [ 107.1972,  309.4759,  220.9165,  428.0770],
        [ 451.6265,  553.7427,  560.8732,  661.7131],
        [ 441.6367,  318.1790,  550.5688,  424.6299],
        [ 445.0392,   88.8733,  546.5665,  191.8780],
        [ 260.7836,  722.6973,  382.2577,  838.5428],
        [ 264.3228,  725.3308,  382.1472,  836.7008],
 

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)
/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


Instances(num_instances=15, image_height=1200, image_width=600, fields=[pred_boxes: Boxes(tensor([[  77.9741,   89.5493,  189.0560,  198.2327],
        [ 227.9717,  251.2907,  338.6708,  363.3689],
        [  64.0426, 1054.7024,  174.7065, 1160.5259],
        [ 220.5112,  493.1717,  341.6058,  609.9316],
        [ 242.9985,   28.3140,  351.6193,  135.8366],
        [  61.7899,  571.4565,  166.4250,  679.9094],
        [  72.5173,  329.3063,  178.9606,  440.0562],
        [ 381.1533,  578.4744,  488.0297,  683.3624],
        [ 205.0098,  720.3322,  322.9976,  839.0151],
        [  58.9052,  821.3240,  165.6692,  927.5817],
        [ 391.1553,  333.7448,  506.4654,  445.8027],
        [ 394.1508,   98.1355,  508.2901,  212.2295],
        [ 373.3104,  804.4122,  471.9643,  903.1134],
        [ 211.8989,  958.5550,  329.5868, 1085.9368],
        [ 211.8501,  963.6202,  329.8938, 1087.4058]])), scores: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1

In [ ]:



img_path = "/workspace/datasets/seg_by_patient/preprocessed/pos_cropped_patch_all_r1_r2/raw1_01-D2(30m)-1.JPG"
# Prepare image
img = cv2.imread(img_path)[:, :, ::-1]  # BGR->RGB

img = minmax_norm(img)

# inputs = predictor.transform_gen.get_transform(img).apply_image(img)
tensor = torch.as_tensor(img.transpose(2,0,1)).cuda().float()

input = [{
    "image":tensor,
    "height": 1200,
    "width": 600
}]

# print(tensor.shape)

# Generate CAM
cam_map = gradcam(input)

# Overlay heatmap
heatmap = cv2.applyColorMap(np.uint8(255 * cam_map), cv2.COLORMAP_JET)

H, W, _ = img.shape

r_hm = cv2.resize(heatmap, (W, H))

r_hm = (r_hm-r_hm.min()) / (r_hm.max()-r_hm.min()+1e-8)

heatmap = cv2.applyColorMap(np.uint8(255 * r_hm), cv2.COLORMAP_JET)


overlay = 0.5 * heatmap[:, :, ::-1] + 0.5 * img
cv2.imwrite("./gradcam_custom.jpg", overlay[:, :, ::-1])

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/module.py:1827: FutureWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  self._maybe_warn_non_full_backward_hook(args, result, grad_fn)


True

In [47]:
heatmap.shape

(38, 19, 3)